In [ ]:
#---------------
# import modules
#---------------

import numpy as np
import cv2
import joblib as jb
import pickle as pk
import matplotlib.pyplot as plt
import mahotas

from os import listdir
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import Isomap
from sklearn.model_selection import train_test_split
from collections import Counter

from sklearn.neural_network import MLPClassifier
#----------------
#Global variables
#----------------

root_path = './dataset2/'
#train_path = 'seg_train/seg_train/'
#test_path = 'seg_test/seg_test/'
#pred_path = 'seg_pred/seg_pred/'

# Dataset classes: buildings, forest, glacier, mountain, sea, street
data_clases = np.array(['cloudy', 
						'rain', 
						'shine', 
						'sunrise'])
IMG_HEIGHT = 150
IMG_WIDTH = 150

#---------------------------------
# Extract Global features
#---------------------------------
bins = 4 

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

#---------------------------------
# prepare training and testing set
#---------------------------------

X_train_aux = []
y_train_aux = []

for c in range(len(data_clases)):
	for filename in listdir(root_path + data_clases[c] + '/'):
		# load image
		img_data = cv2.imread(root_path + data_clases[c] + '/' + filename) 
		img_data = cv2.resize(img_data, (IMG_HEIGHT, IMG_WIDTH))
        # Global Feature extraction
		fv_hu_moments = fd_hu_moments(img_data)
		#print(fv_hu_moments.shape)
		fv_haralick = fd_haralick(img_data)
		#print(fv_haralick.shape)        
		fv_histogram  = fd_histogram(img_data)
		#print(fv_histogram.shape)
		# Concatenate global features
		global_feature = np.hstack([fv_histogram, fv_haralick, fv_hu_moments])
		X_train_aux.append(global_feature)
		y_train_aux.append(c)
		print('> loaded %s %s' % (filename, img_data.shape))
		print('> global features %s %s' % (filename, global_feature.shape))

# convert to array
X_train_aux = np.array(X_train_aux)
y_train_aux = np.array(y_train_aux)

# normalize data
#X_train_aux = X_train_aux / 255

print('X_train shape: ', X_train_aux.shape)
print('y_train shape: ', y_train_aux.shape)

In [3]:
n_particulas = 100
max_iter = 1000
n_training = 10

In [4]:
t = MinMaxScaler()
t.fit(X_train_aux)
X_train_aux = t.transform(X_train_aux)

In [6]:
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X_train_aux, y_train_aux, test_size=0.3, random_state=100)
print(f"Training target statistics: {Counter(y_train_bal)}")
print(f"Testing target statistics: {Counter(y_test_bal)}")

Training target statistics: Counter({3: 253, 0: 196, 2: 179, 1: 157})
Testing target statistics: Counter({0: 104, 3: 104, 2: 72, 1: 57})


In [ ]:
X_sample = len(X_train_bal)
X_input = len(X_train_bal[1])
X_class = len(np.unique(y_train_bal))

X_train = X_train_bal
y_train = y_train_bal
X_test = X_test_bal
y_test = y_test_bal

cost_test = []
metric_train = []

for i in range(n_training):
    print('Training ', i+1, '...')
    clf = MLPClassifier(hidden_layer_sizes=(X_input * 3, ), activation='tanh', solver='adam', alpha=5e-4)
    clf.out_activation_ = 'multiclass'
    partial_loss = []
    
    for _ in range(max_iter):
        clf.partial_fit(X_train, y_train, classes=[0, 1, 2, 3])
        partial_loss.append(clf.loss_)

    metric_train.append(partial_loss)
    y_test_pred = clf.predict(X_test)
    cost_test.append(mean_squared_error(y_test, y_test_pred))
    print ('Test prediction cost with training: ', cost_test[i]) 
    min_cost = min(cost_test)
    print ("Min test prediction cost: ", min_cost)
    if  cost_test[i] <= min_cost:
        # save the model to disk
        filename = 'mcw_model_113_100_1000_84_adam.sav'
        jb.dump(clf, filename)
        np.save('mcw_bestLoss_113_100_1000_84_adam.npy', partial_loss)

with open('mcw_metric_113_100_1000_84_adam', 'wb') as temp:
    pk.dump(metric_train, temp)        

In [8]:
loaded_model = jb.load(filename)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = loaded_model.predict(X_test)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)

y_train_pred_load = loaded_model.predict(X_train)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.2522255192878338  ACC score:  0.8991097922848664
MSE:  0.030573248407643312  ACC score:  0.9923566878980892


In [ ]:
cost_test = []
metric_train = []

for i in range(n_training):
    print('Training ', i+1, '...')
    clf = MLPClassifier(hidden_layer_sizes=(X_input * 3, ), activation='tanh', solver='sgd', alpha=5e-4)
    clf.out_activation_ = 'multiclass'
    partial_loss = []
    
    for _ in range(max_iter):
        clf.partial_fit(X_train, y_train, classes=[0, 1, 2, 3])
        partial_loss.append(clf.loss_)

    metric_train.append(partial_loss)
    y_test_pred = clf.predict(X_test)
    cost_test.append(mean_squared_error(y_test, y_test_pred))
    print ('Test prediction cost with training: ', cost_test[i]) 
    min_cost = min(cost_test)
    print ("Min test prediction cost: ", min_cost)
    if  cost_test[i] <= min_cost:
        # save the model to disk
        filename = 'mcw_model_113_100_1000_84_sgd.sav'
        jb.dump(clf, filename)
        np.save('mcw_bestLoss_113_100_1000_84_sgd.npy', partial_loss)

with open('mcw_metric_113_100_1000_84_sgd', 'wb') as temp:
    pk.dump(metric_train, temp)   

In [10]:
loaded_model = jb.load(filename)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = loaded_model.predict(X_test)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)

y_train_pred_load = loaded_model.predict(X_train)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.228486646884273  ACC score:  0.8931750741839762
MSE:  0.2713375796178344  ACC score:  0.910828025477707


In [ ]:
cost_test = []
loss_train = []

for i in range(n_training):
    print('Training ', i+1, '...')
    clf = MLPClassifier(hidden_layer_sizes=(X_input * 3, ), activation='tanh', solver='lbfgs', max_iter=max_iter, alpha=5e-4)
    clf.out_activation_ = 'multiclass'
    clf.fit(X_train, y_train)
    loss_train.append(clf.loss_)
    y_test_pred = clf.predict(X_test)
    cost_test.append(mean_squared_error(y_test, y_test_pred))
    print ('Test prediction cost with training: ', cost_test[i]) 
    min_cost = min(cost_test)
    print ("Min test prediction cost: ", min_cost)
    if  cost_test[i] <= min_cost:
        # save the model to disk
        filename = 'mcw_model_113_100_1000_84_lbfgs.sav'
        jb.dump(clf, filename)

with open('mcw_metric_113_100_1000_84_lbfgs', 'wb') as temp:
    pk.dump(loss_train, temp)

In [12]:
loaded_model = jb.load(filename)

print("=====================================================================")
print("=====================================================================")
y_test_pred_load = loaded_model.predict(X_test)
cost_test_load = mean_squared_error(y_test, y_test_pred_load)
acc_test_load = accuracy_score(y_test, y_test_pred_load)
print('MSE: ', cost_test_load, ' ACC score: ', acc_test_load)

y_train_pred_load = loaded_model.predict(X_train)
cost_train_load = mean_squared_error(y_train, y_train_pred_load)
acc_train_load = accuracy_score(y_train, y_train_pred_load)
print('MSE: ', cost_train_load, ' ACC score: ', acc_train_load)
print("=====================================================================")
print("=====================================================================")

MSE:  0.19287833827893175  ACC score:  0.9169139465875371
MSE:  0.0  ACC score:  1.0
